<a id="top_page"></a>
# Exoplanet Data and TESS Light Curves Using Python Requests

Exo.mast provides several API tools to obtain the planet parameter information for exoplanets available via that interface. Also, when available you can use these tools to download and plot the TESS light curves.

It is all done via a RESTful API, but it is not available through astroquery. We will show you how to construct the url, do the ***get*** Request and manipulate the resulting information.

[Imports](#imports)<br>
[Get Planet Names](#names)<br>
[Get Planet Properties](#props)<br>
[Get TCE Light Curve](#lightcurve)<br>
[Get TCE Phase Folded Plot](#phased)<br>
[Additional Resources](#additional)<br>
[About This Notebook](#about)<br>


***

<a id="imports"></a>
## Import statements
We need the following python packages to do the imports.
- **numpy** for array manipulation
- **requests** to make the RESTful API get requests
- **matplotlib.pyplot** to plot
- **pandas** to manipulate data
- **IPython.display.display** and **IPython.display.HTML** to render the plot delivered by the API

We also specify two base URLs that we use repeatedly for these requests. The planet url is used to query the planet table.  The Data Validation url is used to query the Data Validation database which holds the header and light curves that are contained in the Data Validation time series files. There is one entry per TCE. TCEs are events (they have a period and an epoch) found in the 2-min cadence data by the TESS Ames Pipeline.  Targets with TCEs have this type of time series which have been flattened and can be easily folded at the period of the reported transit. 

In [ ]:
import numpy as np
import requests
import matplotlib.pyplot as plt
import pandas as p
from IPython.display import display, HTML

%matplotlib inline

planeturl = "https://exo.mast.stsci.edu/api/v0.1/exoplanets/"
dvurl = "https://exo.mast.stsci.edu/api/v0.1/dvdata/tess/"
header={}


<a id="names"></a>
## Query Planet Table for WASP-18b
Get all of the known identifiers for WASP-18 b.  This query takes one parameter, the name of the planet. The output is json which can be converted to a python dictionary. 

In [ ]:
planet_name = "WASP-18 b"

url = planeturl + "/identifiers/"

myparams = {"name":planet_name}

r = requests.get(url = url, params = myparams, headers = header)
print(r.headers.get('content-type'))

If you take a look at the resulting dictionary it includes all the planet Names. It also includes tessID and tessTCE.  These two fields will exist if there is a TCE on this TIC ID with the same period.  We feed these into a variable for later use.

In [ ]:
planet_names = r.json()
ticid = planet_names['tessID']
tce = planet_names['tessTCE']
planet_names

***

<a id="prop"></a>
## Get Planet Properties
Get the planet properties for WASP-18 b.  We hold two exoplanet tables, the composite table at NExScI and Exoplanets.org.  If the planet exists in both tables, you will get back an array of dictionaries where the first element is the NExScI's planet properties and the second is Exoplanet.org's properties.

In [ ]:
url = planeturl + planet_name + "/properties/"

r = requests.get(url = url, headers = header)

planet_prop = r.json()

print("Catalog 1: " + planet_prop[0]['catalog_name'])
print("Catalog 2: " + planet_prop[1]['catalog_name'])
planet_prop[0].keys()

In [ ]:
print("WASP-18 b Properties")
print("Stellar Mass %f %s" % (planet_prop[0]['Ms'], planet_prop[0]['Ms_unit'] ) )
print("Planet Mass %f %s" % (planet_prop[0]['Mp'], planet_prop[0]['Mp_unit'] ) )
print("Planet Mass Reference: %s" % (planet_prop[0]['Mp_ref']))

***

<a id="lightcurve"></a>
## Get TCE  Data and Header for WASP-18 b

First, we provide some background.  TESS does a search of the postage-stamp, two-minute cadence data for transit signals. For every signal it identifies with its Transit Planet Search (TPS) module, it creates something called a Threshold Crossing Event (TCE). TCEs are periodic signals that exceed a nominal signal-to-noise requirement. Some are consistent with transiting planets, others are eclipsing binaries, and others are more consistent with variable stars or noise in the data. The exported products include a data validation time series file that includes a median detrended time series as well as the time series it used to do the planet search.

Here we retrieve the header and data associated with the TCE that corresponds to the ephemeris of WASP-18 b from the data validation time series files. The info request provides the header information from the primary header and the data header for the specified TCE.  Note: this can be done for any TCE, not just those associated with known planets.

Because many TESS targets are observed in more than one sector and the TESS mission does multi-sector searches, we must first determine which sector's search found a TCE on WASP-18. We can then choose from those sector ranges for our query.

In [ ]:
url = dvurl + str(ticid) + '/tces/'
myparams = {"tce" : tce}

r = requests.get(url = url, params = myparams, headers = header)
sectorInfo = r.json()

sectors = [x[:11] for x in sectorInfo["TCE"] if tce in x]
sectors

In [ ]:
url = dvurl + str(ticid) + '/info/'
myparams = {"tce" : tce,
            "sector" : sectors[0]}

r = requests.get(url = url, params = myparams, headers = header)
tceInfo = r.json()
tceInfo.keys()
tceInfo['DV Data Header'].keys()

print("TIC: %s" % tceInfo['DV Primary Header']['OBJECT'])
print("Planet Radius: %f" % tceInfo['DV Data Header']['PRADIUS'])

### Plot the light curve data for WASP-18 b
Use the table request to get the time and median detrended light curve for WASP-18 b. Notice that you can retrieve all the different types of light curves available via this API call.  Some of the more useful ones are:

- LC_DETREND is the median detrended light curve. <br>
- LC_INIT is the initial light curve used to search for the TCE. <br>
- MODEL_INIT is a light curve of the transit model.

In [ ]:
# Get The data
url = dvurl + str(ticid) + '/table/'
myparams = {"tce" : tce,
            "sector" : sectors[0]}

r = requests.get(url = url, params = myparams, headers = header)
tce_data = r.json()

In [ ]:
data = p.DataFrame.from_dict(tce_data['data'])
data.columns

In [ ]:
detrend = data['LC_DETREND']
model = data['MODEL_INIT']
time = data['TIME']

plt.figure(figsize=(14,4))
plt.plot(time,detrend,'.',lw=0.4)
plt.plot(time,model,'r-',lw=0.6)
plt.xlabel('TIME (BTJD)')
plt.ylabel('Relative Flux')

<a id="phased"></a>
## Request a Bokeh Phased Light Curve
The API allows you to pull down a Bokeh plot of the phased light curve, folded and binned at the period of the identified TCE.  Bokeh provides a zoomable, pannable plot to examine. 

In [ ]:
url = dvurl + str(ticid) + '/phaseplot/'
myparams = {"tce" : tce,
            "sector" : sectors[0]}

r = requests.get(url = url, params = myparams, headers = header)

In [ ]:
display(HTML(str(r.content.decode('utf-8'))))

<a id="about_ID"></a>
## About this Notebook
**Author:** Susan E. Mullally, STScI Archive Scientist
<br>**Updated On:** 2018-12-04

[Top of Page](#top_page)
<img style="float: right;" src="https://raw.githubusercontent.com/spacetelescope/notebooks/master/assets/stsci_pri_combo_mark_horizonal_white_bkgd.png" alt="STScI logo" width="200px"/> 